In [16]:
import sys

sys.path.append("../")

# Ultralytics model
from ultralytics import YOLO

# Image reading
import cv2

# OS traversal
import os

# Ploting
import matplotlib.pyplot as plt

# Polygons
from shapely.geometry import Polygon

# Array math
import numpy as np
from pathlib import Path
import yaml
from src.azure_blobs import download_image, get_blobs_by_folder_name
from tqdm import tqdm

# Main funtion to infer and draw snow 
from src.image_processing import predict_snow

# Data wrangling 
import pandas as pd

IMAGE_FORMATS = (".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG")

In [17]:
# Reading the configuraion.yml file 
with open("configuration.yml", "r") as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)

path_to_model = cfg["path_to_model"]
treshold = cfg["treshold"]
padding = cfg["padding"]

# Loading the model
model = YOLO(path_to_model)

In [18]:
# Defining the input and output dirs 
input_dir = os.path.join("input")
output_dir = os.path.join("output") 

# Creating the output dir if it does not exist
os.makedirs(output_dir, exist_ok=True)

# Listing all the images in the input_dir 
images = [f for f in os.listdir(input_dir) if f.endswith(IMAGE_FORMATS)]

# Creating full input paths 
image_paths = [os.path.join(input_dir, image) for image in images]

# Placeholder for predictions 
predictions = []

# Iterating over the images and predicting 
for image_path in tqdm(image_paths):
    # Creating the output name 
    image_name = os.path.basename(image_path)
    image_output_path = os.path.join(output_dir, image_name)

    # Predicting the snow
    mean_val, intersection_found = predict_snow(image_path, image_output_path, model, treshold, padding)

    # Creating a predictions entry 
    predictions.append({"image_name": image_name, "mean_val": mean_val, "intersection_found": intersection_found})

# Creating the dataframe 
predictions_df = pd.DataFrame(predictions)

# Printing the percentage of predictions found 
print(f"Percentage of predictions found: {predictions_df['intersection_found'].mean()}")

# Creating a suquence of 0, 0.1, 0.2, ..., 1 and 
# infering how many images are above the threshold
# for each value in the sequence
thresholds = np.arange(0, 1.1, 0.1)
counts = []
for threshold in thresholds:
    counts.append((predictions_df["mean_val"] > threshold).sum())

# Creating a dataframe with the counts
counts_df = pd.DataFrame({"threshold": thresholds, "count": counts})

# Printing the counts
print(counts_df)

100%|██████████| 63/63 [01:45<00:00,  1.67s/it]

Percentage of predictions found: 0.8412698412698413
    threshold  count
0         0.0     63
1         0.1     63
2         0.2     54
3         0.3     23
4         0.4     17
5         0.5     15
6         0.6     11
7         0.7      2
8         0.8      0
9         0.9      0
10        1.0      0


In [21]:
predictions_df[(predictions_df['mean_val'] > 0.6) & (predictions_df['intersection_found'] == True)]

,image_name,mean_val,intersection_found
14,DJI_0030-2023-12-04-08-51-16.jpg,0.66,True
15,DJI_0032-2023-12-04-08-51-40.jpg,0.61,True
